## Morning practical 2 day 6

Here you'll be burying the traumatic experience of training your own neural networks by doing it _much_ more easily using Keras. We'll work on the same MNIST dataset as before. First run the two code cells below.

In [1]:
#run this cell to set things up
import ipywidgets as widgets, numpy as np, pandas as pd
from numpy.random import default_rng
%matplotlib notebook
import matplotlib.pyplot as plt
import math
import seaborn as sns
from IPython.display import display, Markdown
import sklearn
from sklearn.datasets import make_blobs, make_classification
import itertools

#Keras import
import os
os.environ["KERAS_BACKEND"] = "torch"
import torch
#if Mac Apple silicon
if torch.backends.mps.is_available():
    #fix for MPS 
    os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation, Flatten, Conv2D, MaxPooling2D
from keras.wrappers import SKLearnClassifier

In [2]:
# Read in the data. Liberally copied from: https://keras.io/examples/vision/mnist_convnet/

# # Model / data parameters
# num_classes = 10
# input_shape = (28, 28, 1)

# # the data, split between train and test sets
# (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# # Scale images to the [0, 1] range
# x_train = x_train.astype("float32") / 255
# x_test = x_test.astype("float32") / 255
# # Make sure images have shape (28, 28, 1)
# x_train = np.expand_dims(x_train, -1)
# x_test = np.expand_dims(x_test, -1)
# print("x_train shape:", x_train.shape)
# print(x_train.shape[0], "train samples")
# print(x_test.shape[0], "test samples")


# # convert class vectors to binary class matrices
# y_train = keras.utils.to_categorical(y_train, num_classes)
# y_test = keras.utils.to_categorical(y_test, num_classes)


## Getting a grip on the Keras functional API

We want to remake our fully-connected neural network that we made last week in Keras as a first step. So, let's make sure you understand the Keras functional API. To do so, follow along with [this link](https://keras.io/guides/functional_api/) up to and until Training, evaluation, and inference.

**NOTE: TO BE ABLE TO SHOW THE DIAGRAM, YOU NEED TO RUN conda install pydot IN CONDA (IN YOUR COURSE ENVIRONMENT)**

In [3]:
# your code from following along here. Or if you want it spread over cells, add new ones with esc+b
inputs = keras.Input(shape=(784,))
print(inputs.shape)
dense = layers.Dense(64, activation="relu")
x = dense(inputs)
x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(10)(x)
model = keras.Model(inputs=inputs, outputs=outputs, name="mnist_model")
print(model.summary())
keras.utils.plot_model(model, "my_first_model_with_shape_info.png", show_shapes=True)


(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x_train.reshape(60000, 784).astype("float32") / 255
x_test = x_test.reshape(10000, 784).astype("float32") / 255


rng = np.random.default_rng(seed=42)
train_indices = rng.choice(x_train.shape[0], 20000, replace=False)
test_indices  = rng.choice(x_test.shape[0], 1000, replace=False)

x_train = x_train[train_indices]
y_train = y_train[train_indices]  # Downsample labels as well.
x_test  = x_test[test_indices]
y_test  = y_test[test_indices]

print("Downsampled x_train shape:", x_train.shape)
print("Downsampled x_test shape:", x_test.shape)

print('Got here 1')

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.RMSprop(),
    metrics=["accuracy"],
)

# diagnostic
print('Got here 2')

history = model.fit(x_train, y_train, batch_size=64, epochs=2, validation_split=0.2)

test_scores = model.evaluate(x_test, y_test, verbose=2)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])



(None, 784)


Model: "mnist_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │        50,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 55,050 (215.04 KB)

 Trainable params: 55,050 (215.04 KB)

 Non-trainable params: 0 (0.00 B)

None
Downsampled x_train shape: (20000, 784)
Downsampled x_test shape: (1000, 784)
Got here 1
Got here 2
Epoch 1/2


NotImplementedError: The operator 'aten::_foreach_mul_.Scalar' is not currently implemented for the MPS device. If you want this op to be added in priority during the prototype phase of this feature, please comment on https://github.com/pytorch/pytorch/issues/77764. As a temporary fix, you can set the environment variable `PYTORCH_ENABLE_MPS_FALLBACK=1` to use the CPU as a fallback for this op. WARNING: this will be slower than running natively on MPS.

## Remaking our model from last week

You now basically have all the ingredients to remake your model. In fact, you already made a larger model above. Now, try to do the same thing for the neural network architecture that we used last week, which looked like this:
![NeuralNetworkImage](NeuralNetwork.PNG)
We will use one Hidden Layer with 25 units, and one output layer with 10 units. Use the sigmoid activation that we've used before. See [here](https://keras.io/api/layers/activations/) if you are lost on how to use the sigmoid.

In [ ]:
# your answer here

# answer
#%pdb on
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x_train.reshape(60000, 784).astype("float32") / 255
x_test = x_test.reshape(10000, 784).astype("float32") / 255

print('Got here 1')


ourInputs = keras.Input(shape=(784,))

x = layers.Dense(25, activation="sigmoid")(ourInputs)
ourOutputs = layers.Dense(10, activation="sigmoid")(x)
ourModel = keras.Model(inputs=ourInputs, outputs=ourOutputs, name="mnist_model")

ourModel.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.RMSprop(),
    metrics=["accuracy"],
)

history = ourModel.fit(x_train, y_train, batch_size=64, epochs=2, validation_split=0.2)

test_scores = ourModel.evaluate(x_test, y_test, verbose=2)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])
keras.utils.plot_model(ourModel, "lastWeekModel.png", show_shapes=True)

## What you should see

For one layer and about 80 neurons less you lose 5 percentage points of accuracy. Our simple network is still not too bad. 

## Using the softmax function and ReLUs

Until now, we've been allowing the outputs of our network to sum up to more than one. That's fine for predictions, since we just pick the node with the highest activation and take that as our predicted class. Still, it might be nice, at least for interpretability, if we compress the sum of all these outputs to 1. Then we can interpret each output as a probability of being that class.

What's more, sigmoid neurons have some problems, and Rectified Linear Units (linear, except for negative values which become 0) have been shown to work very well in deep neural networks. Note that they are nonlinear for data < 0, and otherwise linear. That's why they still count as nonlinear activation functions. See [this](https://stats.stackexchange.com/a/510818) or [this](https://www.youtube.com/watch?v=68BZ5f7P94E) for more info (**optional**).

Over to you: use the same network you just trained, only switch to ReLUs and use a softmax output layer! Also, train for 4 instead of 2 epochs (4 full passes over the training data)

In [ ]:
# your answer here

# answer
ourInputs = keras.Input(shape=(784,))

x = layers.Dense(25, activation="relu")(ourInputs)
ourOutputs = layers.Dense(10, activation="softmax")(x)
ourModel = keras.Model(inputs=ourInputs, outputs=ourOutputs, name="mnist_model")

ourModel.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.RMSprop(),
    metrics=["accuracy"],
)

history = ourModel.fit(x_train, y_train, batch_size=64, epochs=4, validation_split=0.2)

test_scores = ourModel.evaluate(x_test, y_test, verbose=2)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])
keras.utils.plot_model(ourModel, "lastWeekModel.png", show_shapes=True)

## What you see

If all goes well, you should see you get a slightly better outcome in this way. This is only due to the ReLUs and/or a different initialisation. I ran it 3 times, and got a worse performance once and a better performance twice. Remember, neural networks get random initialisations to break the symmetry that otherwise occurs in the errors of neurons (causing everything to update the exact same way).

## Training a convolutional neural network
Using Keras we are not beholden to simple dense networks. We can quite easily train a convolutional neural network. Let's do that now and see its performance. Go [here](https://keras.io/examples/vision/mnist_convnet/) and follow the instructions. **Make sure you understand what each layer does, so quickly look up Dropout and Flatten to see what they might do!**. Note that this tutorial doesn't use the functional API but the `Sequential()` syntax. It leads to the same thing, and it is good for you to see that these two different ways of defining Keras models exist!

**Beware: training this network for 15 epochs takes quite some time. Reduce it to 8 epochs instead of the 15 in the example for speed**

In [ ]:
# your answer here


#answer
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


modelConv = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

modelConv.summary()

batch_size = 128
epochs = 15 #change to something like 5 or 7 for speed in your code!

modelConv.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

modelConv.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

score = modelConv.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])



## What you should see

You should see that there is indeed an accuracy of ~99% on the test set. For fewer epochs it might be more ~98%, but still. _Pretty nifty_ for a few lines of code. 

Do note that, if you were to run this code on real handwritten digits, you still need to solve the problem of segmenting out the digits in an image. Also note that in this dataset all the ones look like | and not so much like 1. So a 1 might register as a 7. The point is that this classifier works fine for the narrow task you gave it, so it will perform nigh-on perfectly on classifying MNIST digits, but add a little noise or a small difference, and there's still a problem. It generalises well to the _specific_ problem of MNIST data, but certainly not _as well_ to the actual problem of digit recognition. It is still just a toy classifier. See [here](https://matteo-a-barbieri.medium.com/why-mnist-is-the-worst-thing-that-has-ever-happened-to-humanity-49fd053f0f66) for more info.

## Finally: integrating neural networks with scikit-learn for (nested) cross-validation

What we _want_ to do is to be able to take a neural network and use the tools you have learned for cross-validation to improve it. Ideally, you'd use a scikit-learn Pipeline with a StandardScaler, and then perhaps a dimension reduction step if necessary, and then a neural network, all with tunable hyperparameters, which could all be tuned at once with the pipeline and GridSearchCV or RandomSearchCV. 

Luckily, this is possible. We just need to do two things:
1. Make a _function_ that constructs a Keras neural network model, that takes arguments which are parameters we might want to change. Say, the learning rate, size of Hidden Layer 1, and size of Hidden Layer 2. 
2. Instantiate a scikit-learn model object built from our neural network, using `KerasClassifier`

This is not too difficult. An example of building such a function and instantiating a model is shown below. Make sure you understand the steps: making a model-building function in Keras, wrapping that function in a call that lets it behave as a scikit-learn classifier, and then hyperparameter tuning as normal (here without nested cross-val for speed).

**Note: RandomisedSearchCV depends on random numbers, so your outcome will be slightly different.**

In [ ]:
import numpy as np
from sklearn.model_selection import RandomizedSearchCV

# Set a fixed seed for reproducibility
randomNumberMaker = np.random.default_rng(seed=42)

# Load MNIST data
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.astype("float32") / 255
x_test  = x_test.astype("float32") / 255
x_train = np.expand_dims(x_train, -1)
x_test  = np.expand_dims(x_test, -1)

# Subsample for faster runtime
indicesTrain = randomNumberMaker.integers(0, len(x_train), 3000)
indicesTest  = randomNumberMaker.integers(0, len(x_test), 200)
x_train = x_train[indicesTrain]
y_train = y_train[indicesTrain]
x_test  = x_test[indicesTest]
y_test  = y_test[indicesTest]

def build_neural_net(hiddenLayerOne=784, hiddenLayerTwo=256, learnRate=0.01, **kwargs):
    model = Sequential()
    model.add(Flatten(input_shape=(28, 28, 1)))
    model.add(Dense(hiddenLayerOne, activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(hiddenLayerTwo, activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(10, activation="softmax"))
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learnRate),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

# Create an instance of SKLearnClassifier.
classifier = SKLearnClassifier(
    model=build_neural_net,
    warm_start=False,
    model_kwargs={},             # Initially empty; hyperparameters come from the grid.
    fit_kwargs={'verbose': 0}
)

# Construct a parameter grid with nested model_kwargs.
param_grid = dict(
    model_kwargs=[
        {'hiddenLayerOne': hl1, 'hiddenLayerTwo': hl2, 'learnRate': lr}
        for hl1 in [5, 10, 50]
        for hl2 in [5, 10, 50]
        for lr in [1e-2, 1e-3]
    ]
)

search = RandomizedSearchCV(
    estimator=classifier,
    param_distributions=param_grid,
    scoring="accuracy",
    cv=3,
    n_iter=5,
    n_jobs=-1
)

searchResults = search.fit(x_train, y_train)
bestScore = searchResults.best_score_
bestParams = searchResults.best_params_
print("Best score is {:.2f} using {}".format(bestScore, bestParams))

bestModel = searchResults.best_estimator_
accuracy = bestModel.score(x_test, y_test)
print("Test accuracy: {:.2f}%".format(accuracy * 100))


## What I want you to remember here

There is of course a lot more nuance and power in Keras, this is just the tip of the iceberg. But the practical skills you need to take from here:
* You know how to build and train a neural network model using Keras (both Dense and Convolutional models).
* You know how to wrap a Keras model in a constructor function, and instantiate it as a sklearn model, so that you can use it with sklearn for hyperparameter optimalisation

## The end

With that, the course content is done. All that's left is to apply ML to an example dataset using your understanding of the fundamentals of what's _really_ going on, and the high-level high-powered libraries that you can now command. 

## Survey
[Click me, click me. Oh, oh, oh click me!](https://docs.google.com/forms/d/e/1FAIpQLScURtN2iRNJrLGHqYLxOOZ8Fz8xjoqsIcC_lGzfwuukRvZ0Ew/viewform?usp=sf_link) - after Donkey (on the Shrek DVD main menu)